# 제품 이상여부 판별 프로젝트


## 1. 데이터 불러오기


### 필수 라이브러리


In [10]:
import os
from pprint import pprint

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm

### 데이터 읽어오기


In [11]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [12]:
small_train = train_data.head(10)
small_test = test_data.head(10)

In [13]:
small_train.to_csv('small_train.csv', index=False)
small_test.to_csv('small_test.csv', index=False)

In [14]:
small_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Columns: 464 entries, Wip Line_Dam to target
dtypes: float64(350), int64(77), object(37)
memory usage: 36.4+ KB


In [15]:
small_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Columns: 465 entries, Set ID to target
dtypes: float64(351), int64(77), object(37)
memory usage: 36.5+ KB


In [16]:
import pandas as pd

# CSV 파일 읽기
# small_data = pd.read_csv("train.csv")

def shift_values(data, start_col_index, move_limit):
    for index, row in data.iterrows():
        move_count = 0  # 이동 카운터 초기화
        for col_index in range(start_col_index, data.shape[1]):  # 모든 열을 대상으로
            if pd.isna(row[col_index]) or row[col_index] == "OK":  # 빈값 또는 "OK" 확인
                # 빈값 또는 "OK"가 발견되면 현재 위치부터 이후 3칸 간격의 변수 값을 앞으로 이동
                for shift_index in range(col_index, data.shape[1] - 3, 3):  # 3칸씩 이동
                    # 값을 이동
                    data.iloc[index, shift_index] = data.iloc[index, shift_index + 3]
                    data.iloc[index, shift_index + 3] = None  # 원래 자리 비우기
                    move_count += 1  # 이동 카운트 증가

                    if move_count >= move_limit:  # 설정된 횟수에 도달하면 중지
                        break
            if move_count >= move_limit:  # 외부 루프에서도 체크
                break

# 변수 이름 설정 및 시작 열 인덱스 및 이동 횟수 설정
variables_with_limits = [
    ('HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam', 52),
    ('HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1', 22),
    ('HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2', 22)
]

# 각 변수에 대해 함수 호출
for start_var, move_limit in variables_with_limits:
    start_col_index = small_train.columns.get_loc(start_var)  # 각 변수의 시작 열 인덱스 찾기
    shift_values(small_train, start_col_index, move_limit)

# 수정된 데이터 저장
small_train.to_csv('trim_small_train.csv', index=False)

print('데이터가 성공적으로 수정되고 저장되었습니다.')


데이터가 성공적으로 수정되고 저장되었습니다.


In [17]:
import pandas as pd

# CSV 파일 읽기
# small_data = pd.read_csv("train.csv")

def shift_values(data, start_col_index, move_limit):
    for index, row in data.iterrows():
        move_count = 0  # 이동 카운터 초기화
        for col_index in range(start_col_index, data.shape[1]):  # 모든 열을 대상으로
            if pd.isna(row[col_index]) or row[col_index] == "OK":  # 빈값 또는 "OK" 확인
                # 빈값 또는 "OK"가 발견되면 현재 위치부터 이후 3칸 간격의 변수 값을 앞으로 이동
                for shift_index in range(col_index, data.shape[1] - 3, 3):  # 3칸씩 이동
                    # 값을 이동
                    data.iloc[index, shift_index] = data.iloc[index, shift_index + 3]
                    data.iloc[index, shift_index + 3] = None  # 원래 자리 비우기
                    move_count += 1  # 이동 카운트 증가

                    if move_count >= move_limit:  # 설정된 횟수에 도달하면 중지
                        break
            if move_count >= move_limit:  # 외부 루프에서도 체크
                break

# 변수 이름 설정 및 시작 열 인덱스 및 이동 횟수 설정
variables_with_limits = [
    ('HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam', 52),
    ('HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1', 22),
    ('HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2', 22)
]

# 각 변수에 대해 함수 호출
for start_var, move_limit in variables_with_limits:
    start_col_index = small_test.columns.get_loc(start_var)  # 각 변수의 시작 열 인덱스 찾기
    shift_values(small_test, start_col_index, move_limit)

# 수정된 데이터 저장
small_test.to_csv('trim_small_test.csv', index=False)

print('데이터가 성공적으로 수정되고 저장되었습니다.')


데이터가 성공적으로 수정되고 저장되었습니다.


### 실 작업 코드

In [18]:
import pandas as pd

# CSV 파일 읽기
# small_data = pd.read_csv("train.csv")

def shift_values(data, start_col_index, move_limit):
    for index, row in data.iterrows():
        move_count = 0  # 이동 카운터 초기화
        for col_index in range(start_col_index, data.shape[1]):  # 모든 열을 대상으로
            if pd.isna(row[col_index]) or row[col_index] == "OK":  # 빈값 또는 "OK" 확인
                # 빈값 또는 "OK"가 발견되면 현재 위치부터 이후 3칸 간격의 변수 값을 앞으로 이동
                for shift_index in range(col_index, data.shape[1] - 3, 3):  # 3칸씩 이동
                    # 값을 이동
                    data.iloc[index, shift_index] = data.iloc[index, shift_index + 3]
                    data.iloc[index, shift_index + 3] = None  # 원래 자리 비우기
                    move_count += 1  # 이동 카운트 증가

                    if move_count >= move_limit:  # 설정된 횟수에 도달하면 중지
                        break
            if move_count >= move_limit:  # 외부 루프에서도 체크
                break

# 변수 이름 설정 및 시작 열 인덱스 및 이동 횟수 설정
variables_with_limits = [
    ('HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam', 52),
    ('HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1', 22),
    ('HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2', 22)
]

# 각 변수에 대해 함수 호출
for start_var, move_limit in variables_with_limits:
    start_col_index = test_data.columns.get_loc(start_var)  # 각 변수의 시작 열 인덱스 찾기
    shift_values(test_data, start_col_index, move_limit)

# 수정된 데이터 저장
test_data.to_csv('trim_test_data.csv', index=False)

print('데이터가 성공적으로 수정되고 저장되었습니다.')


데이터가 성공적으로 수정되고 저장되었습니다.
